# Publish an entire folder as an OER "book"

In [ ]:
#!rm -rf bookfiles
rmdir(Path("bookfiles/"))

In [ ]:
from pathlib import Path
import shutil

def rmdir(directory):
    directory = Path(directory)
    for item in directory.iterdir():
        if item.is_dir():
            rmdir(item)
        else:
            item.unlink()
    directory.rmdir()

In [ ]:
#!mkdir -p bookfiles
Path("bookfiles/").mkdir(parents=True, exist_ok=True)

In [ ]:
from pathlib import Path
from jupyterinstruct import InstructorNotebook as inb
from jupyterinstruct.nbfilename import nbfilename
from jupyterinstruct.nbvalidate import validate
from jupyterinstruct import webtools 
from nbconvert.preprocessors import ExecutePreprocessor

In [ ]:
import thisbook

tags = thisbook.tags()

In [ ]:
# jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace $nbfile
# jupyter nbconvert --execute --no-prompt --allow-errors --to html $nbfile
# #	jupyter nbconvert $nbfile --to html
# cp $nbfile $webdir 
# newfile=`basename $nbfile .ipynb`
# cp ${newfile}.html $webdir 

In [ ]:
def makechapter(filename,tags):
    print(filename)
    nb = inb.InstructorNB(filename=filename)

    nb.removebefore('ENDHEADER')
    nb.removeafter('STARTFOOTER')
    nb.headerfooter('Book-Header.ipynb', 'Book-Footer.ipynb')
    nb.removecells('###MSU###')
    studentfile = nb.makestudent(tags,studentfolder="./bookfiles/")
    nb.writenotebook(studentfile)
    validate(studentfile)
   
    #done = webtools.publish2folder(studentfile, website_folder="./bookfiles/")
    #display(done)
    return studentfile

In [ ]:
makechapter(Path('../0-Introduction_INSTRUCTOR.ipynb'),tags);

In [ ]:
#Make a temporary folder and convert all Instructor
#notebooks to student notebooks for testing

directory = Path('..')

allfiles = sorted(directory.glob('*.ipynb'))
allfiles.sort
#Make student versions
for filename in allfiles:
    nbfile = nbfilename(filename)
    if not 'Quiz' in str(nbfile): # Skip quiz prep
        if nbfile.isInstructor: # Only process INSTRUCTOR notebooks
            if nbfile.prefix.isdigit(): # Only process numrical notebooks
                filename = makechapter(filename, tags)

In [ ]:
fid = open('./bookfiles/_toc.yml', 'w')
directory = Path('./bookfiles/')

allfiles = sorted(directory.glob('*.ipynb'))

for ipynb in allfiles:
    if not '0000' in str(ipynb):
        print(ipynb.name)
        fid.write(f'- file: {ipynb.name}\n')

for ipynb in allfiles:
    if '0000' in str(ipynb):
        print(ipynb.name)
        fid.write(f'- file: {ipynb.name}\n')
    
fid.close()

In [ ]:
#!cp Cover_horizontal.png ./bookfiles/
#!cp Cover.jpg ./bookfiles/
#!cp _config.yml ./bookfiles/

import shutil
files = ['Cover_horizontal.png', 'Cover.jpg', '_config.yml']
for f in files:
    shutil.copy(f, 'bookfiles/')

In [ ]:
#Convert TOC to new format
! jupyter-book toc migrate ./bookfiles/_toc.yml -o ./bookfiles/_toc.yml

In [ ]:
!jupyter-book build ./bookfiles/

In [ ]:
#!open ./bookfiles/_build/html/index.html

In [ ]:
#!cp -rf ./bookfiles/_build/html/* ./docs/



def copyDirectory(src, dest):
    try:
        shutil.copytree(src, dest)
    # Directories are the same
    except shutil.Error as e:
        print('Directory not copied. Error: %s' % e)
    # Any error saying that the directory doesn't exist
    except OSError as e:
        print('Directory not copied. Error: %s' % e)

copyDirectory(Path('./bookfiles/_build/html/'), Path('./docs/'))
        

In [ ]:
!open ./docs/index.html